<a href="https://colab.research.google.com/github/Bladefinger/100DaysOfCode/blob/main/beginner_humojahack_fromZindi_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
pd.options.display.float_format = '{:.5f}'.format
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
test = pd.read_csv('test.csv', parse_dates = ['Datetime'])

In [ ]:
# Load files
data = pd.read_csv('train.csv', parse_dates = ['Datetime'])
test = pd.read_csv('test.csv', parse_dates = ['Datetime'])
samplesubmission = pd.read_csv('SampleSubmission.csv')


data.drop(['ID'], axis=1)
# Preview train dataset


,Datetime,Sensor1_PM2.5,Sensor2_PM2.5,Temperature,Relative_Humidity,Offset_fault
0,2021-11-03 04:06:31,52.58000,49.52000,17.40000,96.00000,0
1,2021-11-08 18:43:23,35.25000,33.40000,25.00000,75.00000,0
2,2021-11-07 09:50:33,19.18000,23.50000,24.90000,75.00000,0
3,2022-01-01 18:55:15,19.40000,15.48000,24.90000,70.00000,0
4,2021-11-05 22:23:48,38.30000,34.77000,20.90000,89.00000,0
...,...,...,...,...,...,...
297172,2021-11-19 12:32:00,36.40000,37.77000,28.60000,62.00000,0
297173,2021-11-22 01:22:54,50.12000,40.63000,21.60000,86.00000,1
297174,2021-11-05 01:27:31,53.68000,45.23000,20.20000,94.00000,1
297175,2021-11-23 22:09:30,96.45000,114.67000,20.60000,93.00000,0


In [ ]:
missing_rate = data.isna().sum()/data.shape[0]

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=0)

In [ ]:
trainset = trainset.dropna(axis=0)
testset = testset.dropna(axis=0)

In [ ]:
trainset.isna().sum()/trainset.shape[0]

ID                  0.00000
Datetime            0.00000
Sensor1_PM2.5       0.01229
Sensor2_PM2.5       0.01229
Temperature         0.01181
Relative_Humidity   0.01181
Offset_fault        0.00000
dtype: float64

In [ ]:
X_train = trainset.drop(['ID', 'Datetime', 'Offset_fault'], axis=1)
y_train = trainset['Offset_fault']
X_test = testset.drop(['ID', 'Datetime', 'Offset_fault'], axis=1) 
y_test = testset['Offset_fault']

In [ ]:
test = test.drop(['Datetime'], axis= 1)
#test = test.dropna(axis=0)


In [ ]:
from imblearn.over_sampling import ADASYN

In [ ]:
model = make_pipeline(KNNImputer(n_neighbors=1), PolynomialFeatures(2),SelectKBest(f_classif, k=5),
                      XGBClassifier(random_state=6))

In [ ]:
test_partial = test

In [ ]:
X_train.isna().sum()/X_train.shape[0]

Sensor1_PM2.5       0.01229
Sensor2_PM2.5       0.01229
Temperature         0.01181
Relative_Humidity   0.01181
dtype: float64

In [ ]:
model.fit(X_train, y_train)
 

Pipeline(steps=[('knnimputer', KNNImputer(n_neighbors=1)),
                ('polynomialfeatures', PolynomialFeatures()),
                ('selectkbest', SelectKBest(k=5)),
                ('xgbclassifier', XGBClassifier(random_state=6))])

In [ ]:
ypred = model.predict(X_test)

In [ ]:
test

Index(['ID', 'Sensor1_PM2.5', 'Sensor2_PM2.5', 'Temperature',
       'Relative_Humidity'],
      dtype='object')

In [ ]:
test_partial = test_partial.drop(['ID'], axis=1)

In [ ]:
prediction_test = model.predict(test_partial)

In [ ]:
test_partial_2 = model.transform(test_partial)
#prediction_test_2 = model.predict(test_partial_2)

In [ ]:
print(confusion_matrix(y_test, ypred))
print(classification_report(y_test, ypred))

N, train_score, val_score = learning_curve(model, X_train, y_train,
                                          cv=4, scoring='f1',
                                            train_sizes=np.linspace(0.1, 1, 10))


plt.figure(figsize=(12, 8))
plt.plot(N, train_score.mean(axis=1), label='train score')
plt.plot(N, val_score.mean(axis=1), label='validation score')
plt.legend()

[[34955  2317]
 [ 2898 19266]]
              precision    recall  f1-score   support

           0       0.92      0.94      0.93     37272
           1       0.89      0.87      0.88     22164

    accuracy                           0.91     59436
   macro avg       0.91      0.90      0.91     59436
weighted avg       0.91      0.91      0.91     59436



In [ ]:
from sklearn.model_selection import learning_curve

In [ ]:
id_predit = pd.DataFrame(prediction_test, columns= ['ID'])
#id_predit_2 = pd.DataFrame(prediction_test_2, columns= ['ID'])

In [ ]:
prediction = pd.concat([test["ID"],id_predit], axis=1)
#prediction_2 = pd.concat([test["ID"],id_predit_2], axis=1)

In [ ]:
# # Create a submission file
sub_file = prediction.copy()
sub_file = prediction

#sub_file_2 = prediction_2.copy()
#sub_file_2 = prediction_2
# Create file
sub_file.to_csv('Baseline_7.csv', index = False)
#sub_file_2.to_csv('Baseline__2.csv', index = False)

In [ ]:
B5 = pd.read_csv('Baseline_5.csv')
B6 = pd.read_csv('Baseline_6.csv')

In [ ]:
(B5 == B6).sum()

ID      127361
ID.1    127361
dtype: int64